In [1]:
import duckdb
import numpy as np
from transformers import  DistilBertModel, BigBirdModel, BigBirdTokenizerFast , DistilBertTokenizerFast
from langchain.text_splitter import RecursiveCharacterTextSplitter

2023-09-26 23:40:50.438229: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
con = duckdb.connect(database='jus.duckdb', read_only=False)

In [3]:
# tabela de texto dos juiz
con.execute("""
  CREATE TABLE decisions_vector (
    ID INT PRIMARY KEY,
    content TEXT,
  );
""")

CatalogException: Catalog Error: Table with name "decisions_vector" already exists!

In [ ]:
concedendo_label = 1
negando_label = 0

In [4]:
data = con.execute("SELECT * FROM decisions_description").fetchnumpy()

In [5]:
data["ID"]

array([121184492, 122027196, 123312824, ..., 145089835, 145511648,
       145508379], dtype=int32)

In [6]:
data["content"]

array(['DECISION > It is treated of habeas corpus impetrado for the benefit of Valter de Oliveira Albano - fulfilling penalty in the Center for Resocialization of Lins/SP, currently in closed regime, with the end of the sentence provided for 19/3/2027 -, pointing itself as coactor body maintaining the Court of Justice of São Paulo, which ordered the HC no. 21346-19-77.20 The judgment has this issue (f. 82):<br>HABEAS CORPUS - CRIMINAL EXECUTION - Impetration aiming at granting home imprisonment by integrating risk group, carrier of hypertension - Impossibility - Absence of cabal proof that the prison unit that houses it does not have adequate structure to keep it, nor to offer it,47 if necessary, the appropriate treatment Recommendation No 62 of the CNJ which has no binding character - Implementation of measures To contain the spread of the disease in prison units - Sentenced to serve a closed sentence, which is why it does not fall within the taxable hypotheses provided for in Art. 11

In [7]:
data["summarize"].data

array(['Gulliver argues that the prison system is inherently precarious because it lacks adequate food, medical care, and proper sanitation. The patient, Valter de Albano, has requested for conversion from prison to home, but the prison authorities have rejected his request due to the absence of legally required requirements. Furthermore, the patient\'s condition is not as dire as those described in the national recommendation for release.\nthe authorities decide to take measures against the new encephalovirus in order to reduce the risk of spread and to ensure that the population does not get sick. The authorities also decide to grant house arrest to persons arrested in accordance with the provisions of the Articles of Confederation. According to the news, the prison has good medical assistance and hygiene conditions, but there is no communication from the IoC-19 in the cell. Furthermore, the patient\'s age is young and not in the "age range of risk." This means that the authorities n

In [8]:
data["ID"].size

5805

In [9]:
# Vamos usar numpy.vectorize para aplicar len (comprimento de string) a cada elemento do array
len_vectorized = np.vectorize(len)

# Aplicar a função vetorizada ao array de strings para obter um array de comprimentos
array_len = len_vectorized(data["content"])

# Encontrar o índice do maior comprimento
index_of_max = np.argmax(array_len)

# Usar o índice para encontrar a string mais longa
longest_str = data["content"][index_of_max]

len(longest_str)

186592

In [10]:
tokenizer = BigBirdTokenizerFast.from_pretrained("google/bigbird-roberta-large") # large
model = BigBirdModel.from_pretrained("google/bigbird-roberta-large",attention_type="block_sparse")

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, chunk_size=4096, chunk_overlap=10 #86683 4096
)

In [11]:
texts = text_splitter.split_text(longest_str)
print(len(texts))

inputs = tokenizer(texts[0], return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

print(last_hidden_states.shape)

13
torch.Size([1, 3806, 1024])


In [ ]:
con.close()